In [5]:
import pandas as pd

data = pd.read_csv('Data/Cleaned/Integrated_data.csv')
data['date'] = pd.to_datetime(data['date'])

# Create the list of dates and mean glucose values
date_list = data[['date', 'mean_glucose']].dropna().sort_values(by='date')

date_list.head()

,date,mean_glucose
11,2024-03-23,114.419355
12,2024-03-24,107.410256
13,2024-03-25,99.793103
14,2024-03-26,100.968421
15,2024-03-27,109.431818


In [6]:
# Calculate the moving average of 3 days
date_list['mean_glucose_avg3'] = date_list['mean_glucose'].rolling(window=3).mean()

date_list.head()

,date,mean_glucose,mean_glucose_avg3
11,2024-03-23,114.419355,NaN
12,2024-03-24,107.410256,NaN
13,2024-03-25,99.793103,107.207572
14,2024-03-26,100.968421,102.723927
15,2024-03-27,109.431818,103.397781
